In [1]:
import pandas as pd
import numpy as np

In [2]:
s1920 = pd.read_csv(r'C:\Users\George\Desktop\football-match-prediction-using-ml-techniques\data\Bundesliga\D1-1920.csv')

understats = pd.read_csv(r'C:\Users\George\Desktop\football-match-prediction-using-ml-techniques\data\understat_per_game.csv')

In [3]:
s1920 = s1920[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR','HTHG','HTAG',
              'HS', 'AS', 'HST','AST','HC','AC', 'HF', 'AF', 'B365H', 'B365A', 'B365D',
               'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA']].reset_index(drop=True)

In [4]:
s1920.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,...,B365D,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA
0,16/08/2019,Bayern Munich,Hertha,2,2,D,1,2,17,6,...,8.00,1.19,7.73,15.30,1.18,7.5,15.00,1.18,8.0,15.0
1,17/08/2019,Dortmund,Augsburg,5,1,H,1,1,23,5,...,7.00,1.23,6.76,13.52,1.20,6.5,15.00,1.22,6.5,15.0
2,17/08/2019,Freiburg,Mainz,3,0,H,0,0,19,19,...,3.25,2.23,3.45,3.44,2.20,3.3,3.40,2.20,3.4,3.4
3,17/08/2019,Leverkusen,Paderborn,3,2,H,2,2,13,11,...,6.00,1.29,6.07,10.57,1.27,5.8,11.00,1.29,6.0,10.5
4,17/08/2019,Werder Bremen,Fortuna Dusseldorf,1,3,A,0,1,23,12,...,3.75,1.75,4.05,4.68,1.73,3.9,4.75,1.75,4.0,4.6


In [5]:
s1920['Date'] = pd.to_datetime(s1920['Date'],format="%d/%m/%Y")

In [6]:
understats['Date'] = pd.to_datetime(understats['date']).dt.strftime("%Y-%m-%d")
understats['Date'] = pd.to_datetime(understats['Date'])

In [7]:
understats['Date']

0       2014-08-22
1       2014-08-30
2       2014-09-13
3       2014-09-20
4       2014-09-23
           ...    
24575   2020-07-15
24576   2020-07-19
24577   2020-07-22
24578   2020-07-26
24579   2020-07-29
Name: Date, Length: 24580, dtype: datetime64[ns]

In [8]:
understats =understats.replace({'team':{'Hamburger SV':'Hamburg','Bayer Leverkusen':'Leverkusen','Eintracht Frankfurt':'Ein Frankfurt',
                           "Borussia M.Gladbach":"M'gladbach",'Hertha Berlin':'Hertha','Mainz 05':'Mainz',
                           'Hannover 96':'Hannover','Borussia Dortmund':'Dortmund','VfB Stuttgart':'Stuttgart',
                           'FC Cologne':'FC Koln', 'RasenBallsport Leipzig':'RB Leipzig','Nuernberg':'Nurnberg',
                           'Fortuna Duesseldorf':'Fortuna Dusseldorf'}})

In [9]:
df = pd.merge(s1920,understats, how='inner',left_on=['Date','HomeTeam'],right_on=['Date','team'])

In [10]:
df.shape

(306, 57)

In [11]:
df.rename(columns = {'xG':"HxG", 'xGA':'AxG','deep':'Hdeep','deep_allowed':'Adeep',
                     'ppda_coef':'Hppda_coef','oppda_coef':'Appda_coef'},inplace=True)

df = df.drop(['npxG','npxGA','scored', 'missed', 'xpts', 'result', 'wins',
              'draws', 'loses', 'pts', 'npxGD', 'ppda_att', 'ppda_def',
              'oppda_att', 'oppda_def', 'team', 'xG_diff', 'xGA_diff',
              'xpts_diff', 'date', 'year','h_a'],axis=1)

In [12]:
# Calculating the average of betting sites odd for each result
avgH = round((df['B365H'] + df['PSH'] + df['WHH'] + df['VCH'])/4,2)
avgD = round((df['B365D'] + df['PSD'] + df['WHD'] + df['VCD'])/4,2)
avgA = round((df['B365A'] + df['PSA'] + df['WHA'] + df['VCA'])/4,2)

df['AVGH'] = avgH
df['AVGD'] = avgD
df['AVGA'] = avgA

#drop the existing columns with betting odds
df = df.drop(['PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 
              'VCD', 'VCA', 'B365H', 'B365D', 'B365A'],axis=1)

In [13]:
df['season'] = '2019/2020'

In [14]:
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,...,HxG,AxG,Hdeep,Adeep,Hppda_coef,Appda_coef,AVGH,AVGD,AVGA,season
0,2019-08-16,Bayern Munich,Hertha,2,2,D,1,2,17,6,...,2.47707,1.184590,17,2,10.500000,16.217391,1.17,7.81,15.08,2019/2020
1,2019-08-17,Dortmund,Augsburg,5,1,H,1,1,23,5,...,3.25906,0.904482,14,3,5.913043,22.666667,1.21,6.69,14.13,2019/2020
2,2019-08-17,Freiburg,Mainz,3,0,H,0,0,19,19,...,1.88401,0.784216,3,8,20.200000,10.391304,2.22,3.35,3.41,2019/2020
3,2019-08-17,Leverkusen,Paderborn,3,2,H,2,2,13,11,...,2.47615,1.204750,9,2,3.838710,17.379310,1.27,5.97,11.02,2019/2020
4,2019-08-17,Werder Bremen,Fortuna Dusseldorf,1,3,A,0,1,23,12,...,2.01120,1.015360,14,7,11.866667,21.000000,1.74,3.92,4.70,2019/2020


In [15]:
df.head(9)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,...,HxG,AxG,Hdeep,Adeep,Hppda_coef,Appda_coef,AVGH,AVGD,AVGA,season
0,2019-08-16,Bayern Munich,Hertha,2,2,D,1,2,17,6,...,2.477070,1.184590,17,2,10.500000,16.217391,1.17,7.81,15.08,2019/2020
1,2019-08-17,Dortmund,Augsburg,5,1,H,1,1,23,5,...,3.259060,0.904482,14,3,5.913043,22.666667,1.21,6.69,14.13,2019/2020
2,2019-08-17,Freiburg,Mainz,3,0,H,0,0,19,19,...,1.884010,0.784216,3,8,20.200000,10.391304,2.22,3.35,3.41,2019/2020
3,2019-08-17,Leverkusen,Paderborn,3,2,H,2,2,13,11,...,2.476150,1.204750,9,2,3.838710,17.379310,1.27,5.97,11.02,2019/2020
4,2019-08-17,Werder Bremen,Fortuna Dusseldorf,1,3,A,0,1,23,12,...,2.011200,1.015360,14,7,11.866667,21.000000,1.74,3.92,4.70,2019/2020
5,2019-08-17,Wolfsburg,FC Koln,2,1,H,1,0,14,11,...,0.979300,0.662001,5,2,8.555556,7.103448,1.91,3.79,3.94,2019/2020
6,2019-08-17,M'gladbach,Schalke 04,0,0,D,0,0,16,8,...,1.431410,0.760773,7,5,5.757576,15.285714,2.06,3.48,3.70,2019/2020
7,2019-08-18,Ein Frankfurt,Hoffenheim,1,0,H,1,0,17,10,...,1.821400,0.634831,4,4,11.454545,6.444444,2.30,3.66,2.98,2019/2020
8,2019-08-18,Union Berlin,RB Leipzig,0,4,A,0,3,13,14,...,0.782907,3.933870,2,12,15.769231,8.227273,4.78,4.03,1.71,2019/2020


In [16]:
df.loc[df['HomeTeam']=='Bayern Munich','ranking_HT_ls'] = 1
df.loc[df['HomeTeam']=='Dortmund','ranking_HT_ls'] = 2
df.loc[df['HomeTeam']=='RB Leipzig','ranking_HT_ls'] = 3
df.loc[df['HomeTeam']=='Leverkusen','ranking_HT_ls'] = 4
df.loc[df['HomeTeam']=="M'gladbach",'ranking_HT_ls'] = 5
df.loc[df['HomeTeam']=='Wolfsburg','ranking_HT_ls'] = 6
df.loc[df['HomeTeam']=='Ein Frankfurt','ranking_HT_ls'] = 7
df.loc[df['HomeTeam']=='Werder Bremen','ranking_HT_ls'] = 8
df.loc[df['HomeTeam']=='Hoffenheim','ranking_HT_ls'] = 9
df.loc[df['HomeTeam']=='Fortuna Dusseldorf','ranking_HT_ls'] = 10
df.loc[df['HomeTeam']=='Hertha','ranking_HT_ls'] = 11
df.loc[df['HomeTeam']=='Mainz','ranking_HT_ls'] = 12
df.loc[df['HomeTeam']=='Freiburg','ranking_HT_ls'] = 13
df.loc[df['HomeTeam']=='Schalke 04','ranking_HT_ls'] = 14
df.loc[df['HomeTeam']=='Augsburg','ranking_HT_ls'] = 15
df.loc[df['HomeTeam']=='FC Koln','ranking_HT_ls'] = -1
df.loc[df['HomeTeam']=='Paderborn','ranking_HT_ls'] = -2
df.loc[df['HomeTeam']=='Union Berlin','ranking_HT_ls'] = -3

In [17]:
df.loc[df['AwayTeam']=='Bayern Munich','ranking_AT_ls'] = 1
df.loc[df['AwayTeam']=='Dortmund','ranking_AT_ls'] = 2
df.loc[df['AwayTeam']=='RB Leipzig','ranking_AT_ls'] = 3
df.loc[df['AwayTeam']=='Leverkusen','ranking_AT_ls'] = 4
df.loc[df['AwayTeam']=="M'gladbach",'ranking_AT_ls'] = 5
df.loc[df['AwayTeam']=='Wolfsburg','ranking_AT_ls'] = 6
df.loc[df['AwayTeam']=='Ein Frankfurt','ranking_AT_ls'] = 7
df.loc[df['AwayTeam']=='Werder Bremen','ranking_AT_ls'] = 8
df.loc[df['AwayTeam']=='Hoffenheim','ranking_AT_ls'] = 9
df.loc[df['AwayTeam']=='Fortuna Dusseldorf','ranking_AT_ls'] = 10
df.loc[df['AwayTeam']=='Hertha','ranking_AT_ls'] = 11
df.loc[df['AwayTeam']=='Mainz','ranking_AT_ls'] = 12
df.loc[df['AwayTeam']=='Freiburg','ranking_AT_ls'] = 13
df.loc[df['AwayTeam']=='Schalke 04','ranking_AT_ls'] = 14
df.loc[df['AwayTeam']=='Augsburg','ranking_AT_ls'] = 15
df.loc[df['AwayTeam']=='FC Koln','ranking_AT_ls'] = -1
df.loc[df['AwayTeam']=='Paderborn','ranking_AT_ls'] = -2
df.loc[df['AwayTeam']=='Union Berlin','ranking_AT_ls'] = -3

In [18]:
df.to_csv('bundesliga-1920.csv', index=False)